In [13]:
import numpy as np
import pandas as pd
import joblib

In [14]:
df=pd.read_csv("train.csv")
df.columns.values

array(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination',
       'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa',
       'VRDeck', 'Name', 'Transported'], dtype=object)

In [15]:
df.iloc[0,:]

PassengerId             0001_01
HomePlanet               Europa
CryoSleep                 False
Cabin                     B/0/P
Destination         TRAPPIST-1e
Age                        39.0
VIP                       False
RoomService                 0.0
FoodCourt                   0.0
ShoppingMall                0.0
Spa                         0.0
VRDeck                      0.0
Name            Maham Ofracculy
Transported               False
Name: 0, dtype: object

In [16]:
df.isnull().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [17]:
df.columns

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Name', 'Transported'],
      dtype='object')

In [18]:
['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Name', 'Transported'],

(['PassengerId',
  'HomePlanet',
  'CryoSleep',
  'Cabin',
  'Destination',
  'Age',
  'VIP',
  'RoomService',
  'FoodCourt',
  'ShoppingMall',
  'Spa',
  'VRDeck',
  'Name',
  'Transported'],)

In [19]:
df['Destination'].unique()

array(['TRAPPIST-1e', 'PSO J318.5-22', '55 Cancri e', nan], dtype=object)

In [20]:
test_df=pd.read_csv("test.csv")
test_df.columns.values
test_df.isnull().sum()
test_df.columns

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Name'],
      dtype='object')

In [25]:
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.impute import SimpleImputer
import pandas as pd
import numpy as np
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import StandardScaler
class ExperimentalTransformer(BaseEstimator,TransformerMixin):
    def __init__(self):
        pass
    def fit(self,X,y=None):
        return self
    def transform(self,X,y=None):
        
        #A=X[['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age','VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']]
        imp_mf = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
        Home_Planet=X['HomePlanet'].array.reshape(-1, 1)
        imp_mf.fit(Home_Planet)
        Home_Planet_transformed=imp_mf.transform(Home_Planet)
        Home_Planet_imputed_final=pd.DataFrame(Home_Planet_transformed,columns=['HomePlanet'])
        Home_Planet_imputed_final.replace(to_replace=['Europa','Earth','Mars'], value=[1,2,3],inplace=True)
 
        Cryo_Sleep=X['CryoSleep']
        if(Cryo_Sleep.isnull().sum()>0):
            imp_mf2 = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
            Cryo_Sleep=X['CryoSleep'].array.reshape(-1, 1)
            imp_mf2.fit(Cryo_Sleep)
            Cryo_Sleep_transformed=imp_mf2.transform(Cryo_Sleep)
            Cryo_Sleep_imputed_final=pd.DataFrame(Cryo_Sleep_transformed,columns=['CryoSleep'])
            Cryo_Sleep_imputed_final = Cryo_Sleep_imputed_final.astype(int)
        else:
            Cryo_Sleep_imputed_final=Cryo_Sleep.astype(int)
            
            
        imp_mf3 = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
        Destination=X['Destination'].array.reshape(-1, 1)
        imp_mf3.fit(Destination)
        Destination_transformed=imp_mf3.transform(Destination)
        Destination_transformed_imputed_final=pd.DataFrame(Destination_transformed,columns=['HomePlanet'])        
        Destination_transformed_imputed_final.replace(to_replace=['TRAPPIST-1e', 'PSO J318.5-22', '55 Cancri e'], value=[1,2,3],inplace=True)       
        VIP=X['VIP']
        if(VIP.isnull().sum()>0):
            imp_mf4 = SimpleImputer(missing_values=np.nan, strategy='most_frequent')   
            VIP=X['VIP'].array.reshape(-1, 1)
            imp_mf4.fit(VIP)
            VIP_transformed=imp_mf4.transform(VIP)
            VIP_imputed_final=pd.DataFrame(VIP_transformed,columns=['VIP'])
            VIP_imputed_final = VIP_imputed_final.astype(int)
        else:
            VIP_imputed_final=VIP.astype(int)     
        IQR =X['Age'].quantile(0.75)-X['Age'].quantile(0.25)
        value=X['Age'].quantile(0.75)+(1.5*IQR)
        Age_final=X['Age']=X['Age'].fillna(value)
        rest=X[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']]

        for var in rest:
            IQR =rest[var].quantile(0.75)-rest[var].quantile(0.25)
            value=rest[var].quantile(0.75)+(1.5*IQR)
            rest[var]=rest[var].fillna(value)
        features = pd.concat([Home_Planet_imputed_final, Cryo_Sleep_imputed_final,Destination_transformed_imputed_final,Age_final,VIP_imputed_final,rest], axis=1)
        transformer = FunctionTransformer(np.sqrt)
        transformer.fit(features)
        X_transformed=transformer.transform(features)
        scaler=StandardScaler()
        Z= scaler.fit_transform(X_transformed)
        

    

        

        return Z

        
        


In [26]:
y=df['Transported']

y = y.astype(int)


In [23]:
# filename = 'preprocessing_model.sav'
# joblib.dump(obj, filename)

In [27]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
model= RandomForestClassifier(n_estimators = 1000)
obj = ExperimentalTransformer()
obj.transform(df)

X=obj.transform(df)
grid2 = {'max_depth':[10,15,300]}
bestmodel = GridSearchCV(model, grid2, cv = 5)
bestmodel.fit(X, y)
print(bestmodel.score(X,y))
from sklearn.linear_model import LogisticRegression
log_model=LogisticRegression(penalty = 'l2', max_iter = 100000,random_state=42,C=10)

grid1 = {'solver':['lbfgs', 'newton-cg', 'sag']}

bestLogReg = GridSearchCV(log_model, grid1, cv= 5)
bestLogReg.fit(X, y)
print(bestLogReg.score(X,y))

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier

bag_clf=BaggingClassifier(DecisionTreeClassifier(max_depth=20),n_estimators=10,max_samples=800,bootstrap=True)
bag_clf.fit(X,y)
print(bag_clf.score(X,y))


from sklearn import svm
clf = svm.SVC(kernel='linear')

clf.fit(X, y)
print(clf.score(X,y))


from sklearn.ensemble import VotingClassifier
voting_clf=VotingClassifier(estimators=[('rf',bestmodel),('lr',log_model),('svc',clf),('dt',bag_clf)],voting='hard')
voting_clf.fit(X,y)
print(voting_clf.score(X,y))


/home/arijit/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/arijit/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


0.8375704589899919
0.7879903370528011
0.8038651788795582
0.7889106177384102
0.814448406764063


In [28]:
filename = 'model.sav'
joblib.dump(voting_clf, filename)

['model.sav']

In [29]:
test_df=pd.read_csv("test.csv")
test_df.columns.values
test_df.isnull().sum()
test_df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez


In [30]:
obj= ExperimentalTransformer()
obj.transform(test_df)

/home/arijit/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


array([[ 0.11435947,  1.33044268, -0.61194577, ..., -0.51210587,
        -0.49390451, -0.45864603],
       [ 0.11435947, -0.75162953, -0.61194577, ..., -0.51210587,
         2.93401347, -0.45864603],
       [-1.55941783,  1.33044268,  1.8559391 , ..., -0.51210587,
        -0.49390451, -0.45864603],
       ...,
       [ 1.39869396,  1.33044268,  1.8559391 , ..., -0.51210587,
        -0.49390451, -0.45864603],
       [-1.55941783, -0.75162953, -0.61194577, ..., -0.51210587,
        -0.49390451,  0.97827968],
       [ 0.11435947,  1.33044268,  0.78444827, ..., -0.51210587,
        -0.49390451, -0.45864603]])

In [31]:
filename1 = 'preprocessing.sav'
joblib.dump(obj, filename1)

['preprocessing.sav']

In [32]:
loaded_preprocessing = joblib.load('preprocessing.sav')
result = loaded_preprocessing.transform(test_df)
print(result)

[[ 0.11435947  1.33044268 -0.61194577 ... -0.51210587 -0.49390451
  -0.45864603]
 [ 0.11435947 -0.75162953 -0.61194577 ... -0.51210587  2.93401347
  -0.45864603]
 [-1.55941783  1.33044268  1.8559391  ... -0.51210587 -0.49390451
  -0.45864603]
 ...
 [ 1.39869396  1.33044268  1.8559391  ... -0.51210587 -0.49390451
  -0.45864603]
 [-1.55941783 -0.75162953 -0.61194577 ... -0.51210587 -0.49390451
   0.97827968]
 [ 0.11435947  1.33044268  0.78444827 ... -0.51210587 -0.49390451
  -0.45864603]]


/home/arijit/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [33]:
loaded_model = joblib.load(filename)
result = loaded_model.predict(loaded_preprocessing.transform(test_df))
print(result)

/home/arijit/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


[1 0 1 ... 1 1 1]


In [34]:
test_df_1=pd.read_csv("test_1.csv")
test_df_1


,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0087_01,Earth,False,E/3/P,TRAPPIST-1e,19,False,0,20,135,265,367,Meremy Jarvey,True


In [35]:
result_1 = loaded_model.predict(loaded_preprocessing.transform(test_df_1))
result_1[0]

/home/arijit/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


0